In [3]:
import sys
import os
import numpy as np

NENSMEMBERS=20
NPARAMETERS=6
PRIOR_ERR=0.5
DA_DIR = '/gws/nopw/j04/jules/ndouglas/FluxNet/u-dr501/da_files'

ALPHA_PRIOR=0.068
NEFF_PRIOR=0.008
OMEGA_PRIOR=0.15
TLOW_PRIOR=5.0
TUPP_PRIOR=37.5
G1_PRIOR=3.8

def test_bnds(xbi, val_bnds):

    for xi in enumerate(xbi):
        if val_bnds[xi[0]][0] <= xi[1] <= val_bnds[xi[0]][1]:
            continue
        else:
            return False
    return True


def generate_param_ens(param_dic, size_ens=10, prior_err=0.25):

    p_keys = list(param_dic.keys())
    xb = []
    val_bnds = []
    for param in p_keys:
        xb.append(param_dic[param][0])
        val_bnds.append(param_dic[param][1])
    
    xb = np.array(xb)
    xb_sd = xb * prior_err
    b_mat = np.eye(len(xb))*((xb_sd)**2)

    ens = []
    i = 0
    while len(ens) < size_ens:
        xbi = np.random.multivariate_normal(xb, b_mat)
        i += 1
        if test_bnds(xbi, val_bnds) is True:
            ens.append(xbi)
        else:
            continue
    return np.array(ens)


if __name__ == '__main__':

    prior_params = {
            'a_alpha_io': [ALPHA_PRIOR, (0.04, 0.096)],
            'b_neff_io': [NEFF_PRIOR, (0.001, 0.015)],
            'c_omega_io': [OMEGA_PRIOR, (0.1, 0.2)],
            'd_tlow_io': [TLOW_PRIOR, (0.0,10.0)],
            'e_tupp_io': [TUPP_PRIOR, (30.0, 45.0)],
            'f_g1_stomata_io':[G1_PRIOR, (1.6, 6.0)]
            }
    
    param_ens = generate_param_ens(prior_params, size_ens=NENSMEMBERS,
				prior_err=PRIOR_ERR)
    ens_mean=np.mean(param_ens,0)

    np.save(DA_DIR+'/Xb_'+str(NENSMEMBERS), np.transpose(param_ens))
    np.save(DA_DIR+'/xb_'+str(NENSMEMBERS), np.transpose(ens_mean))
